In [1]:
import numpy as np
import time
import mujoco
import mujoco.viewer

In [60]:
modelfilename = 'test4n.xml'
m = mujoco.MjModel.from_xml_path(modelfilename)
d = mujoco.MjData(m)
mrec = [[] for _ in range(45)]
print(m.dof_armature)
print(len(m.dof_armature))
sinamp = 0.1
fs = 0
# m.dof_armature[6:]=[10]*14

def mycontroller(m,d):
    # dofd = [10]*14
    # m.dof_armature[6:] = dofd
    global fs
    rls = 0
    lls = 0
    
    mujoco.mj_forward(m, d)
    rfid = mujoco.mj_name2id(m,mujoco.mjtObj.mjOBJ_BODY,'link3')
    lfid = mujoco.mj_name2id(m,mujoco.mjtObj.mjOBJ_BODY,'link6')
    rhid = mujoco.mj_name2id(m,mujoco.mjtObj.mjOBJ_BODY,'link1') #Composite_HIPRY
    lhid = mujoco.mj_name2id(m,mujoco.mjtObj.mjOBJ_BODY,'link4') #Composite_HIPLY
    rfpos = d.xpos[rfid].copy()
    lfpos = d.xpos[lfid].copy()
    rhip  = d.xpos[rhid].copy()
    lhip  = d.xpos[lhid].copy()
    vecr2l = lhip - rhip
    vecr2lxynorm = np.sqrt(vecr2l[0]*vecr2l[0] + vecr2l[1]*vecr2l[1])
    fRy = np.array([vecr2l[0]/vecr2lxynorm,vecr2l[1]/vecr2lxynorm,0])
    fRz = np.array([0,0,1])
    fRx = np.cross(fRy,fRz)
    fRT = np.array([fRx,fRy,fRz])
    fp = vecr2l/2 + rhip
    fpi = -fRT@fp
    
    h = 1
    g = 9.8
    omega = np.sqrt(g/h)
    mujoco.mj_subtreeVel(m,d)
    vcom = d.subtree_linvel[1].copy()
    com  = d.subtree_com[1].copy()
    eta = com + vcom/omega
    frfpos = fRT@rfpos + fpi
    flfpos = fRT@lfpos + fpi
    fcom = fRT@com + fpi
    fvcom = fRT@vcom
    fvp = fRT@(vcom/omega)
    feta = fcom + fvp
    
    if d.sensordata[0]<0.01 and d.sensordata[1]<0.01 and d.sensordata[2]<0.01 and d.sensordata[3]<0.01:
        rls = 1
    if d.sensordata[4]<0.01 and d.sensordata[5]<0.01 and d.sensordata[6]<0.01 and d.sensordata[7]<0.01:
        lls = 1
    
    if feta[1]-frfpos[1] > 0.21 and fs == 0: #eta[1] > 0.08 #0.07
        pass
        ###########################################
        print('time:',d.time,'fs_state_transition:0->1')
        fs = 1
    if flfpos[1]-feta[1] > 0.21 and fs == 1: #eta[1] < -0.08
        pass
        ###########################################
        print('time:',d.time,'fs_state_transition:1->0')
        fs = 0
        
    kp = 80
    kd = 9
    kp2 = kp
    kd2 = kd
    kp3 = 5
    kd3 = 3
    theta = 0
    initald = 10
    #参数说明：关节名称    Kp  位置目标  Kd     速度目标 D
    cl = {'HIPRY'  :[   kp,     0,  kd,     0,  initald],
          'HIPRX'  :[   kp, theta,  kd,     0,  initald],
          'KNEERY' :[   kp,     0,  kd,     0,  initald],
          'HIPRZ'  :[   kp,     0,  kd,     0,  initald],
          'ANKLERY':[   kp2,    0,  kd2,    0,  initald],
          'ANKLERX':[   kp3,    0,  kd3,    0,  initald],
          'HIPLY'  :[   kp,     0,  kd,     0,  initald],
          'HIPLX'  :[   kp, theta,  kd,     0,  initald],
          'HIPLZ'  :[   kp,     0,  kd,     0,  initald],
          'KNEELY' :[   kp,     0,  kd,     0,  initald],
          'ANKLELY':[   kp2,    0,  kd2,    0,  initald],
          'ANKLELX':[   kp3,    0,  kd3,    0,  initald],
          'WAISTY' :[   kp,     0,  kd,     0,  initald],
          'WAISTX' :[   kp,     0,  kd,     0,  initald]}
        
    #knee:0.7 ank:0.12 
    output = 0
    hipyor = 0
    hipyol = 0
    apitr = 0
    apitl = 0
    kneer = 0
    kneel = 0
    hipzr = 0
    hipzl = 0
    kneeliftangle = 0.7 #0.7 #1.5 0.7
    amoo = 0.08 #0.08
    swingang = 0.1
    anklebias = 0.05 #0.08 $0.16 #0.06
    pos2ang = 0.9
    posbias = -0.0 #0也行
    hipb = 0.0 #0.1 0.12
    liftratio = 0
    b = 0.2 #0.2
    # b = 0.2*sin(d.time*2*np.pi/6)+0.25
    # if d.time <4: #4
    #     b = 0.1
    # else:
    #     b = 0.4 #0.4
    mrec[23].append(b)
    if fs == 0:
        # t1.setCPNL(t1.fclist)
        output = -amoo
        kneel = kneeliftangle
        diffpos = fcom[0] + b*fvcom[0] - frfpos[0]#feta[0] - frfpos[0] + posbias
        ang = diffpos#diffpos*pos2ang
        kneel = ang*0 + kneeliftangle
        if lls == 1:
            if abs(diffpos) >= 2:
                print('error!!!!!!')
            ang = np.arcsin(diffpos/2)
            # ang = swingang
            hipyor = ang
            hipyol = -ang
            # hipzr = -0.5
        hipyol += -kneeliftangle/2+hipb+ang*liftratio
        apitl += -anklebias-kneeliftangle/2-ang*liftratio-hipb
        apitr += -anklebias 
        if fvcom[1] > 0:
            kneel = 0
            hipyol += kneeliftangle/2-hipb-ang*liftratio
            apitl += kneeliftangle/2+ang*liftratio+hipb
        pass
    elif fs == 1:
        # t1.setCPNL(t1.fclist)
        output = amoo
        kneer = kneeliftangle
        diffpos = fcom[0] + b*fvcom[0] - flfpos[0]#feta[0] - flfpos[0] + posbias
        ang = diffpos#diffpos*pos2ang
        kneer = ang*0 + kneeliftangle
        if rls == 1:
            if abs(diffpos) >= 2:
                print('error!!!!!!')
            ang = np.arcsin(diffpos/2)
            # ang = swingang
            hipyor = -ang
            hipyol = ang
            # hipzl = 0.5
        hipyor += -kneeliftangle/2+hipb+ang*liftratio
        apitl += -anklebias
        apitr += -anklebias-kneeliftangle/2-ang*liftratio-hipb
        if fvcom[1] < 0:
            kneer = 0
            hipyor += kneeliftangle/2-hipb-ang*liftratio
            apitr += kneeliftangle/2+ang*liftratio+hipb
        pass
    else:
        pass
        # raise ValueError('fs has bad value.')
        
    cl['HIPRX'][1]  = output
    cl['HIPLX'][1]  = output
    cl['HIPRY'][1]  = hipyor
    cl['HIPLY'][1]  = hipyol
    cl['KNEERY'][1] = kneer
    cl['KNEELY'][1] = kneel
    cl['ANKLERY'][1]= apitr
    cl['ANKLELY'][1]= apitl
    
    for jn,tm in cl.items():
        m.dof_armature[m.jnt(jn).id+5] = tm[4]
        d.ctrl[m.jnt(jn).id-1] = (-tm[0]*(d.qpos[m.jnt(jn).id+6]-tm[1]) -tm[2]*(d.qvel[m.jnt(jn).id+5]-tm[3]))*tm[4]

def set_cam_viewer(m, viewer):
    # 设置镜头跟随
    viewer.cam.type = mujoco.mjtCamera.mjCAMERA_TRACKING
    viewer.cam.trackbodyid = mujoco.mj_name2id(m, mujoco.mjtObj.mjOBJ_BODY, "link0")
    viewer.cam.distance = 3.0# 设置摄像机距离 
    viewer.cam.elevation = -20# 设置摄像机仰角
    
    # 设置摄像机为固定点模式
    # viewer.cam.type = mujoco.mjtCamera.mjCAMERA_FIXED
    # 设定摄像机的位置和朝向
    # viewer.cam.lookat = [0, 0, 0]  # 摄像机视线的目标点
    # viewer.cam.distance = 5.0      # 摄像机与目标点的距离
    # viewer.cam.elevation = -20     # 摄像机仰角
    # viewer.cam.azimuth = 45        # 摄像机方位角

with mujoco.viewer.launch_passive(m, d) as viewer:
  # set_cam_viewer(m, viewer)
  # Close the viewer automatically after 30 wall-seconds.
  start = time.time()
  ii = 0
#   qst = pin.Quaternion(pin.AngleAxis(0,np.array([0,0,1])).matrix()).coeffs().copy()
#   qslt = qst[3]
#   qst[3] = qst[2]
#   qst[2] = qst[1]
#   qst[1] = qst[0]
#   qst[0] = qslt
#   d.qpos[3:7] = qst
  mujoco.mj_forward(m, d)
  while viewer.is_running() and time.time() - start < 25:
    step_start = time.time()

    if ii % 5 == 0:
        mycontroller(m,d)
    ii += 1
    

    mrec[24].append(d.ctrl[0])
    mrec[25].append(d.ctrl[3])
    mrec[26].append(d.ctrl[4])
    
    # mj_step can be replaced with code that also evaluates
    # a policy and applies a control signal before stepping the physics. 
    mujoco.mj_step(m, d)
    mrec[1].append(d.subtree_com[1][0])
    mrec[2].append(d.subtree_com[1][2])
    mrec[3].append(d.subtree_com[1][1])
    mrec[4].append(sinamp*np.sin(d.time - 2.5)) #这里也要跟着改
    angmomm = np.zeros((3,m.nv))
    mujoco.mj_angmomMat(m,d,angmomm,0)
    hhere = angmomm@d.qvel
    mrec[5].append(hhere[0])
    mrec[6].append(hhere[1])
    mrec[7].append(hhere[2])
    # mrecord8.append(d.qpos[8])
    # mrecord9.append(d.qpos[14])
    mujoco.mj_subtreeVel(m,d)
    mrec[30].append(d.subtree_linvel[1][0])
    mrec[31].append(d.subtree_linvel[1][1])
    mrec[32].append(d.subtree_linvel[1][2])

    # Example modification of a viewer option: toggle contact points every two seconds.
    with viewer.lock():
      viewer.opt.flags[mujoco.mjtVisFlag.mjVIS_CONTACTPOINT] = int(d.time % 2)

    # Pick up changes to the physics state, apply perturbations, update options from GUI.
    viewer.sync()

    # Rudimentary time keeping, will drift relative to wall clock.
    time_until_next_step = m.opt.timestep - (time.time() - step_start)
    if time_until_next_step > 0:
      time.sleep(time_until_next_step)

[0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0.]
20
time: 0.16500000000000012 fs_state_transition:0->1
time: 0.5900000000000004 fs_state_transition:1->0
time: 1.259999999999972 fs_state_transition:0->1
time: 1.9999999999998905 fs_state_transition:1->0
time: 2.6749999999998164 fs_state_transition:0->1
time: 3.349999999999742 fs_state_transition:1->0
time: 4.0299999999996805 fs_state_transition:0->1
time: 4.714999999999909 fs_state_transition:1->0
time: 5.40500000000014 fs_state_transition:0->1
time: 6.09500000000037 fs_state_transition:1->0
time: 6.790000000000602 fs_state_transition:0->1
time: 7.485000000000834 fs_state_transition:1->0
time: 8.180000000000906 fs_state_transition:0->1
time: 8.870000000000523 fs_state_transition:1->0
time: 9.56000000000014 fs_state_transition:0->1
time: 10.249999999999758 fs_state_transition:1->0
time: 10.939999999999376 fs_state_transition:0->1
time: 11.63499999999899 fs_state_transition:1->0
time: 12.329999999998606 fs_state_transition:0->